# Trend Analysis on Twitter live data using Spark Streaming

Import sparkContext & StreamingContext from PySpark library.

In [1]:
from __future__ import print_function
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

Create a sparkContext with AppName "SteramingTwitterAnalysis".<br>
Setting the LogLevel of SparkContext to ERROR. This will not print all the logs which are INFO or WARN level.<br>
Create Spark Streaming Context using SC (spark context). parameter 10 is the batch interval. <br>
Every 10 second the analysis will be done.

In [2]:
sc = SparkContext(appName="SteramingTwitterAnalysis")
sc.setLogLevel("ERROR")
ssc = StreamingContext(sc, 10)

Connect to socket broker using ssc (spark streaming context)<br>
Host : "172.31.20.58" (localhost) & port : 5555

In [3]:
socket_stream = ssc.socketTextStream("172.28.2.154", 5555)

window function parameter sets the Window length. All the analysis will be done on tweets stored for 60 secs.

In [4]:
lines = socket_stream.window( 20 )

### Process the Stream:
1. Receives tweet message, stored in lines. **Input DStream**
2. splits the messages into words. **Apply transformation on DStream : flatMap**
3. filters all the words which start with a hashtag(#). **transformation : filter**
4. converts the words to lowercase. **transformation : map**
5. maps each tag to (word, 1). **transformation : map**
6. then reduces and counts occurrences of each hash tag. (action : reduceByKey) hashtags = **output DStream**

In [5]:
hashtags = lines.flatMap( lambda text: text.split( " " ) ).filter( lambda word: word.lower().startswith("#") ).map( lambda word: ( word.lower()
, 1 ) ).reduceByKey( lambda a,b:a+b)

Sort the hashtags based on the counts in decreasing order

In [6]:
author_counts_sorted_dstream = hashtags.transform(lambda foo:foo.sortBy(lambda x:x[0].lower()).sortBy(lambda x:x[1],ascending=False))

Print the final analysis: Most popular hashtags on streaming twitter data

In [7]:
author_counts_sorted_dstream.pprint()

### Starting the Spark Streaming:
Spark Streaming code we have written till now will not execute, untill we start the ssc.<br>
ssc.start() will start the spark streaming context. This is the Action for the whole code. <br>
Now it'll create the lineage & DAG & do the lazy evaluation & start running the whole sequesnce of code.


In [8]:
ssc.start()

awaitTermination() is very important to stop the SSC.<br> 
When we kill this python process then this signal will be sent to awaitTermination() function.<br> 
it will finally stop the spark streaming job.

In [9]:
ssc.awaitTermination()

Py4JJavaError: An error occurred while calling o22.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyspark\streaming\util.py", line 68, in call
    r = self.func(t, *rdds)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyspark\streaming\dstream.py", line 633, in <lambda>
    self.func = lambda t, rdd: func(t, prev_func(t, rdd))
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyspark\streaming\dstream.py", line 297, in <lambda>
    func = lambda t, rdd: oldfunc(rdd)
  File "<ipython-input-6-1b3177cf4a8c>", line 1, in <lambda>
    author_counts_sorted_dstream = hashtags.transform(lambda foo:foo.sortBy(lambda x:x[0].lower()).sortBy(lambda x:x[1],ascending=False))
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyspark\rdd.py", line 699, in sortBy
    return self.keyBy(keyfunc).sortByKey(ascending, numPartitions).values()
  File "C:\ProgramData\Anaconda3\l